# Dask processing tests

this needs graphviz for visualizations

In [ ]:
import dask
import dask.array as da
import numpy as np


## define a computational graph
embarassingly parallel here 

In [ ]:
def f(x):
    return x + 1

def g(x):
    return x * 2

def h(x):
    return x - 3

def m(x):  
    return da.mean(x)  # we can also use numpy.mean here if we want, but the latter will directly materialize the array. da.mean needs a 'compute' call to materialize the array instead

def identity(x):
    return x

In [ ]:
f_delayed = dask.delayed(f)
g_delayed = dask.delayed(g)
h_delayed = dask.delayed(h)
m_delayed = dask.delayed(m)

In [ ]:
input_placeholder = dask.delayed(identity, pure = True)(da.arange(2))
fx = f_delayed(input_placeholder)
gx = g_delayed(fx)
hx = h_delayed(gx)
mx = m_delayed(hx)
input_key = input_placeholder.key
computation = mx 
graph = dict(computation.dask)

In [ ]:
input_key

In [ ]:
computation

In [ ]:
graph

In [ ]:
dask.visualize(computation, filename='dask_graph.svg', format='svg')
# Save the graph to a file

### Execute with dask

replace input with something more supstantial

In [ ]:
og_data = np.random.rand(int(1e9))

wrap numpy array into dask array. only for the experiments here

In [ ]:
input_data = da.from_array(og_data, chunks=(int(1e6),))

built the result by hand to be able to check it 

replace input of graph with new input and compute result 

In [ ]:
dask.config.set(scheduler='synchronous') # make ALL operations synchronous
graph[input_key] = input_data 
result = dask.get(graph, mx.key,).compute()
result

try with parallel scheduler

In [ ]:
dask.config.set(scheduler='processes') 

graph[input_key] = input_data 
result = dask.get(graph, mx.key).compute()  # Use processes scheduler
result

for dataframes, have a look [here](https://examples.dask.org/dataframe.html), for xarray, check [this](https://docs.xarray.dev/en/stable/user-guide/dask.html) out, and [that](https://examples.dask.org/xarray.html?highlight=schedulers) 

## Partial parallelization of a computational graph with `dask.annotate`

annotate `h` to be serial only

In [ ]:

def h_single(x): 
    with dask.config.set(scheduler='synchronous'):
        hx =  h(x)
    return hx.rechunk('auto') # re-split the data again to make downstream operations parallel

h_delayed_serial = dask.delayed(h_single)

build up the graph again

In [ ]:
input_placeholder = dask.delayed(identity, pure = True)(da.arange(2))
fx = f_delayed(input_placeholder)
gx = g_delayed(fx)
hx = h_delayed_serial(gx) # README: use the serial version of h
mx = m_delayed(hx)
input_key = input_placeholder.key
computation = mx 
graph = dict(computation.dask) 

execute the thing again with a parallel scheduler - check how the CPU load behaves.

In [ ]:
dask.config.set(scheduler='processes')  # make operations parallel
graph[input_key] = input_data
result = dask.get(graph, computation.key).compute()  # Use processes scheduler
result

## Custom dask graph 

In [ ]:
import dask

In [ ]:
def add(x, y):
    return x + y

def multiply(x, y):
    return x * y

def affine(x, a = 1, b = 0):
    return a * x + b

def save_data(x, filename):
    np.save(filename, x)

def read_data(filename):
    return np.load(filename)

In [ ]:
import dask.delayed

tasks = {
    "read": dask.delayed(read_data),
    "add":dask.delayed(add),
    "multiply": dask.delayed(multiply), 
    "affine": dask.delayed(affine),
    "save": dask.delayed(save_data),
}

read_node = tasks['read']("input_data.npy")
add_node = tasks['add'](read_node, 5)
multiply_node = tasks['multiply'](add_node, read_node)
affine_node = tasks['affine'](multiply_node, a=2, b=3)
save_data_node = tasks['save'](affine_node, "output_data.npy")

In [ ]:
import numpy as np
input_data = np.random.rand(100) 
np.save("input_data.npy", input_data)

In [ ]:
save_data_node.visualize()

In [ ]:
save_data_node.compute()

In [ ]:
output_data = np.load("output_data.npy")
output_data

In [ ]:
2 * ((input_data + 5 ) * input_data ) + 3 